In [ ]:
import random
import time
import Statistics as st

# Inject noise into the data part of the frame
def noisychannel(frame):
    data_part = list(frame[-st.CRC_SIZE - st.DATA_SIZE: -st.CRC_SIZE])
    # Flip random bits in the data part
    for index in random.sample(range(len(data_part)), random.randint(0, len(data_part) - 1)):
        data_part[index] = '1' if data_part[index] == '0' else '0'
    # Rebuild the frame
    return frame[:st.N_SIZE + st.LENGTH_SIZE] + ''.join(data_part) + frame[-st.CRC_SIZE:]

# Introduce random delay (up to 5 seconds)
def delay():
    time.sleep(random.randint(0, 5))

# XOR operation between two binary strings
def xor(a, b):
    return ''.join('0' if x == y else '1' for x, y in zip(a, b))

# Perform binary division for CRC calculation
def binary_division(dividend, divisor):
    rem = dividend[:len(divisor)]
    for bit in dividend[len(divisor):]:
        rem = xor(rem, divisor)[1:] if rem[0] == '1' else rem[1:]
        rem += bit
    return rem

# CRC-4 ITU calculation
def crc4itu(frame):
    return binary_division(frame + '0000', "10011")

# Create a frame with sequence number and data
def makeFrame(n, data):
    return (f"{str(n).zfill(st.N_SIZE)}{str(len(data)).zfill(st.LENGTH_SIZE)}"
            f"{data.zfill(st.DATA_SIZE)}{crc4itu(data).zfill(st.CRC_SIZE)}")

# Extract details from received frame
def receiveFrame(frame):
    n = int(frame[:st.N_SIZE])
    l = int(frame[st.N_SIZE:st.N_SIZE + st.LENGTH_SIZE])
    data = frame[-st.CRC_SIZE - l: -st.CRC_SIZE]
    crc = frame[-st.CRC_SIZE:] if data != 'q' else ""
    return n, l, data, crc
